# Bundesrat Repraesentation

Analyse der Bundesräte zwischen 1848 und 2019 auf Basis folgender Faktoren:
- Herkunft - familiär (vermögend/akademisch)
- Geschlecht
- Alter
- Parteizugehörigkeit
- Herkunft - geographisch (Kanton / Region)

---

In [2]:
# import libraries
import pandas as pd
import numpy as np
from datetime import datetime
import os

In [1000]:
# import list of federal council from wikipedia de and wikipedia en
link = "https://de.wikipedia.org/wiki/Liste_der_Mitglieder_des_Schweizerischen_Bundesrates"
bundesraete = pd.read_html(link,header=0)[0]
link = "https://en.wikipedia.org/wiki/List_of_members_of_the_Swiss_Federal_Council"
federal_council = pd.read_html(link, header=0)[0]

In [1001]:
# merge the two wikipedia tables
council_merge = pd.merge(bundesraete, federal_council[["Name", "Election", "Left office", "Party", "Seq[6]"]], on="Name", how="left")

In [1002]:
# check for cases where the merge on name did'nt work
council_merge[pd.isna(council_merge.Election)]

,Jahr,Bild,Name,Lebensdaten,Kanton,Partei,Election,Left office,Party,Seq[6]
9,1855–1875,NaN,Josef Martin Knüsel,1813–1889,Luzern,freis.,NaN,NaN,NaN,NaN
35,1900–1912,NaN,Marc Ruchet,1853–1912,Waadt,FDP,NaN,NaN,NaN,NaN


In [1003]:
# fix data by hand
council_merge.loc[council_merge.Name == "Josef Martin Knüsel", "Election"] = "14 July 1855"
council_merge.loc[council_merge.Name == "Josef Martin Knüsel", "Left office"] = "31 December 1875"
council_merge.loc[council_merge.Name == "Josef Martin Knüsel", "Party"] = "FDP/PRD"
council_merge.loc[council_merge.Name == "Josef Martin Knüsel", "Seq[6]"] = "D02"

council_merge.loc[council_merge.Name == "Marc Ruchet", "Election"] = "14 December 1899"
council_merge.loc[council_merge.Name == "Marc Ruchet", "Left office"] = "9 July 1912"
council_merge.loc[council_merge.Name == "Marc Ruchet", "Party"] = "FDP/PRD"
council_merge.loc[council_merge.Name == "Marc Ruchet", "Seq[6]"] = "G05"

In [1004]:
# transform dates to datetime format
council_merge.Election = council_merge.Election.apply(lambda x: datetime.strptime(x, "%d %B %Y"))
council_merge["Left office"] = council_merge["Left office"].apply(lambda x: "amtierend" if (x == "Incumbent") else datetime.strptime(x.split("[")[0], "%d %B %Y"))

In [1005]:
# drop unused columns
council_merge.drop(["Bild", "Partei"], axis=1, inplace=True)

In [4]:
# Export data to edit manually (don't execute anymore!)
#### council_merge.to_csv('internal/data_edit.csv', index=False)

In [3]:
# import modified data & clean data
council_merge = pd.read_csv('internal/data_edit.csv')
council_merge.Election = council_merge.Election.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
council_merge["Left office"] = council_merge["Left office"].apply(lambda x: datetime.today() if (x == "amtierend") else datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
council_merge['birthyear'] = council_merge['Lebensdaten'].apply(lambda x: x[0:4] if len(x) > 8 else x[2:6])
council_merge['seat'] = council_merge['Seq'].str[0:1]
council_merge['Entered office'] = council_merge.Jahr.apply(lambda x: datetime.strptime(x[0:4], '%Y'))
council_merge['family'] = (council_merge.Reichtum == 1) | (council_merge['Akad-Familie'] == 1)

---

### Zusammenstellung des Bundesrats für einzelne Jahre ermitteln

In [4]:
# calculate year with wrong number of BRs
for year in range(1848,2019):
    temp = council_merge[(council_merge['Entered office'] <= '{}-11-30'.format(year)) & ((council_merge['Left office'] > '{}-10-30'.format(year)))]
    if (len(temp) != 7):
        print("{}: {}".format(year, len(temp)))

1867: 8
1893: 6
1902: 6
1983: 6
1995: 8
2009: 8
2010: 9
2017: 8


In [5]:
### fix data
# 1867
council_merge.loc[15, "Entered office"] = datetime(1868, 1, 1, 0, 0)
# 1893
council_merge.loc[25, "Left office"] = datetime(1893, 12, 30, 0, 0)
# 1902
council_merge.loc[27, "Left office"] = datetime(1902, 12, 30, 0, 0)
# 1983
council_merge.loc[83, "Left office"] = datetime(1983, 12, 30, 0, 0)
# 1984
council_merge.loc[90, "Left office"] = datetime(1984, 12, 30, 0, 0)
council_merge.loc[93, "Entered office"] = datetime(1985, 1, 1, 0, 0)
# 1995
council_merge.loc[100, "Entered office"] = datetime(1996, 1, 1, 0, 0)
# 2009
council_merge.loc[111, "Entered office"] = datetime(2010, 1, 1, 0, 0)
# 2010
council_merge.loc[112, "Entered office"] = datetime(2011, 1, 1, 0, 0)
council_merge.loc[113, "Entered office"] = datetime(2011, 1, 1, 0, 0)
# 2017
council_merge.loc[116, "Entered office"] = datetime(2018, 1, 1, 0, 0)


In [6]:
# create year-by-year dataset and calculate summary stats
data = pd.DataFrame()
for i, year in enumerate(range(1848,2020)):
    data.loc[i,"year"] = year
    br_nr = 1
    br_age = 0
    temp = council_merge[(council_merge['Entered office'] <= '{}-11-30'.format(year)) & ((council_merge['Left office'] > '{}-10-30'.format(year)))]
    for j, member in temp.iterrows():
        data.loc[i,"BR_{}".format(member['Seq'][0:1])] =  member['Name']
        br_nr += 1
        br_age += year - int(member['birthyear'])
    data.loc[i, 'mean_age'] = br_age / (br_nr-1)
    data.loc[i, 'nr_parties'] = len(temp.Party.unique())
    data.loc[i, 'nr_acad'] = temp.Akademiker.sum()
    data.loc[i, 'nr_family'] = temp.family.sum()
    data.loc[i, 'nr_jobs'] = len(temp.Beruf.unique())
    data.loc[i, 'share_female'] = 100 * (1 - (temp.Geschlecht.sum() / 7))
    for party in temp.Party.unique():
        data.loc[i, 'share_{}'.format(party)] = 100 * (len(temp[temp.Party == party]) / 7)

---

## Altersklassen
Basierend auf Volkszählungs- und BFS-Daten

In [7]:
# read age trend
alter = pd.read_excel("input/B.08a.xlsx", skiprows=3)
alter = alter[3:17]

In [8]:
# rername columns
alter.columns = ['Jahr', '2.5', '7.5', '12.5', '17.5', '22.5', '27.5',
       '32.5', '37.5', '42.5', '47.5', '52.5', '57.5',
       '62.5', '67.5', '72.5', '77.5', '82.5', '87.5',
       'Total', 'Annee']

In [9]:
# calculate mean age for given years
age_chunks = pd.DataFrame()
for i, row in alter[['Jahr','2.5', '7.5', '12.5', '17.5', '22.5', '27.5',
       '32.5', '37.5', '42.5', '47.5', '52.5', '57.5',
       '62.5', '67.5', '72.5', '77.5', '82.5', '87.5']].iterrows():
    year = row['Jahr'] 
    row = row.drop(labels=['Jahr'])
    total = row.sum()
    calc = 0
    for j, number in row.iteritems():
        calc += float(j)*float(number)
    age_chunks.loc[i, "Jahr"] = str(year)[0:4]
    age_chunks.loc[i, "Mean_age"] = calc/total

In [10]:
# create list of yearly data (starting 1860)
age_years = pd.DataFrame()
for i, year in enumerate(range(1860,1991)):
    age_years.loc[i, "Jahr"] = year
    if age_chunks.Jahr.isin(['{}'.format(int(year))]).any():
        age_years.loc[i, "age_population"] = age_chunks[age_chunks.Jahr == str(year)]["Mean_age"].values[0]

In [11]:
# interpolate missing years
age_years.age_population = age_years.age_population.interpolate()

In [12]:
# import bfs data (https://www.pxweb.bfs.admin.ch/pxweb/de/px-x-0102020000_103/px-x-0102020000_103/px-x-0102020000_103.px)
age_years_bfs = pd.read_csv("input/px-x-0102020000_103.csv", encoding = "ISO-8859-1", delimiter=";", skiprows=1)

In [13]:
# clean data
age_years_bfs['Alter'] = age_years_bfs['Alter'].apply(lambda x: x.split(" Jahr")[0])
age_years_bfs = age_years_bfs.drop(columns=["Demografische Komponente", "Staatsangehörigkeit (Kategorie)", "Geschlecht", 'Alter'])

In [14]:
# transpose dataframe
age_years_bfs = age_years_bfs.transpose()

In [15]:
# calculate mean age for each year
for i, row in age_years_bfs.iterrows():
    total = row.sum()
    age_total = 0
    for j, grp in row.iteritems():
        if j == 99:
            j = 102.5
        age_total += j * grp
    age_years_bfs.loc[i, "mean_age"] = age_total / total

In [16]:
# prepare data for merge
age_years_bfs = age_years_bfs[["mean_age"]].reset_index()
age_years_bfs.columns = ["year", "age_population"]
age_years.columns = ["year", "age_population"]
age_years = age_years[:111]

In [17]:
# append bfs data to census data
age_years = age_years.append(age_years_bfs)

In [18]:
# change column type
age_years.year = age_years.year.astype('float')

In [19]:
# merge age data with main year-by-year dataset
data = pd.merge(data, age_years, how='left', on="year")

---

## Parteistärken
Parteistärken aufgrund der historischen Sitzverteilungen im Nationalrat ermitteln und Index erstellen, der beschreibt, wie gut der Bundesrat diese Machteverhältnisse repräsentiert

In [20]:
# import historical data
parteistaerke = pd.read_excel("input/X.06.xlsx", skiprows=16, header=None)

In [21]:
# rename columns
parteistaerke.columns = ["Jahr", "FDP/PRD", "LPS/PLS", "CVP/PDC", "Konservative", "EVP", "soz.-pol., Demokr",
                         "SPS/PSS", "SVP/UDC", "Kommunisten / PdA.", "LdU", "div.", "Nation. Aktion / SD",
                         "POCH / GPS", "Autopartei", "Lega", "Total", "(leer)", "SR_FDP/PRD", "SR_LPS/PLS", "SR_CVP/PDC",
                         "SR_Konservative", "SR_soz.-pol., Demokr", "SR_SPS/PSS", "SR_SVP/UDC", "SR_LdU",
                         "SR_div.", "SR_Total"]

In [22]:
# import BFS and clean data (from 1919 onwards) (https://www.bfs.admin.ch/bfs/de/home/statistiken/politik/wahlen/nationalratswahlen.assetdetail.11048413.html)
parteistaerke_bfs = pd.read_excel("input/je-d-17.02.02.02.01.01.xlsx", skiprows=1, index_col=0)
parteistaerke_bfs = parteistaerke_bfs[:25]
parteistaerke_bfs = parteistaerke_bfs.transpose().reset_index()
parteistaerke_bfs['index'] = parteistaerke_bfs['index'].apply(lambda x: str(x)[0:4])

In [23]:
# rename columns
parteistaerke_bfs.columns = ['index', 'Total', "FDP/PRD", "CVP/PDC", "SPS/PSS", "SVP/UDC", 'Dem. ', 'LPS/PLS',
       'LdU', 'EVP', 'CSP', 'GLP', 'BDP/PBD', 'PdA ', 'PSA ', 'POCH', 'GPS',
       'FGA 4)', 'Sol. 5)', 'SD', 'Rep. 6)', 'EDU', 'FPS', 'Lega', 'MCR',
       'Übrige 7)']

In [24]:
# prepare datasets for merge
parteistaerke['BDP/PBD'] = 0
parteistaerke_bfs['BDP/PBD'] = parteistaerke_bfs['BDP/PBD'].apply(lambda x: str(x).replace("*", "0"))
parteistaerke_bfs['LPS/PLS'] = parteistaerke_bfs['LPS/PLS'].apply(lambda x: str(x).replace("*", "0"))
parteistaerke_bfs['LPS/PLS'] = parteistaerke_bfs['LPS/PLS'].apply(lambda x: str(x).replace("-> FDP", "0"))
parteistaerke = parteistaerke[["Jahr", "FDP/PRD", "CVP/PDC", "SPS/PSS", "SVP/UDC", 'BDP/PBD', 'LPS/PLS', "Total"]]
parteistaerke_bfs = parteistaerke_bfs[['index', "FDP/PRD", "CVP/PDC", "SPS/PSS", "SVP/UDC", 'BDP/PBD', 'LPS/PLS', 'Total']]
parteistaerke = parteistaerke[:23]
parteistaerke_bfs.columns = ["Jahr", "FDP/PRD", "CVP/PDC", "SPS/PSS", "SVP/UDC", 'BDP/PBD', 'LPS/PLS', "Total"]

In [25]:
# append bfs data to historical data
parteistaerke = parteistaerke.append(parteistaerke_bfs, ignore_index=True)
parteistaerke = parteistaerke.fillna(0)

In [26]:
# calculcate share for each BR-party
for col in ["FDP/PRD", "CVP/PDC", "SPS/PSS", "SVP/UDC", 'BDP/PBD', 'LPS/PLS']:
    for i, row in parteistaerke.iterrows():
        parteistaerke.loc[i, "population_share_{}".format(col)] = 100 / int(row['Total']) * int(row[col])

In [27]:
# subset data
parteistaerke = parteistaerke[['Jahr', 'population_share_FDP/PRD', 'population_share_CVP/PDC',
       'population_share_SPS/PSS', 'population_share_SVP/UDC',
       'population_share_BDP/PBD', 'population_share_LPS/PLS']]
parteistaerke['Jahr'] = parteistaerke['Jahr'].apply(int)

In [28]:
# expand data to each year (inluding years inbetween elections)
partei_years = pd.DataFrame()
index_list = []
for year in range(1848,2020):
    years = parteistaerke.Jahr.tolist()
    if (year in years):
        jahr = year
    elif (year-1 in parteistaerke.Jahr.tolist()):
        jahr = year-1
    elif (year-2 in parteistaerke.Jahr.tolist()):
        jahr = year-2
    elif (year-3 in parteistaerke.Jahr.tolist()):
        jahr = year-3

    partei_years = partei_years.append(parteistaerke[parteistaerke.Jahr == jahr][['Jahr', 'population_share_FDP/PRD', 'population_share_CVP/PDC',
       'population_share_SPS/PSS', 'population_share_SVP/UDC',
       'population_share_BDP/PBD', 'population_share_LPS/PLS']], ignore_index=True)
    index_list.append(year)
partei_years['year'] = index_list

In [29]:
# merge parties with main year-by-year dataset
data = pd.merge(data, partei_years, how='left', on='year')

In [30]:
# replace NAs with 0
data = data.fillna(0)

In [31]:
# calculate:
## yearly difference of share in BR and share in NR for each BR-party (in case the party was part of the BR in the respective year)
for i, party in enumerate(['FDP/PRD', 'CVP/PDC', 'BDP/PBD', 'SVP/UDC', 'SPS/PSS', 'LPS/PLS']):
    difference = data["share_{}".format(party)] - data["population_share_{}".format(party)]
    for j, val in data["share_{}".format(party)].iteritems():
        if val == 0:
            difference[j] = 0
    ## store absolute value of this difference divided by 100 -> leads to a number between 0 and 1
    data["diff_{}".format(party)] = (abs(difference) / 100)

In [32]:
# summarize all previously calculated differences
data["BR_closeness"] = data[["diff_FDP/PRD", "diff_CVP/PDC", "diff_BDP/PBD", "diff_SVP/UDC", "diff_SPS/PSS", 'diff_LPS/PLS']].sum(axis=1)
# subtract this sum from 1 -> 1: good representation; 0: bad representation
data["BR_closeness"] = 1-data["BR_closeness"]

---

# Kantonale Vertretung
Brechnung der kantonalen / regionalen Vertretung im Bundesrat seit 1848. Zusätzlich: Differenz ermitteln zu der Anzahl Sitze und Tage die dem Kanton / der Region gemäss Einwohnerzahlen 2018 zugestanden hätten.

### Kantone

In [33]:
# import list of cantons from wikipedia
link = "https://de.wikipedia.org/wiki/Kanton_(Schweiz)"
kantone = pd.read_html(link,header=0)[0]

In [34]:
# import population count from BFS
kantone_bfs = pd.read_excel("input/je-d-21.03.02.xlsx", skiprows=3, index_col=0)
kantone_bfs = kantone_bfs.drop(columns=["Jahre", "Schweiz"])[3:4].transpose().reset_index()
kantone_bfs.columns = ["Abk.", "Einwohner"]
kantone_bfs.Einwohner = kantone_bfs.Einwohner * 1000

In [35]:
# merge imported datasets
kantone = pd.merge(kantone, kantone_bfs, how="left", on="Abk.")

In [36]:
# only keep necessary columns
kantone = kantone[['Abk.', 'Kanton', 'Einwohner', 'Bei­tritt']]

In [37]:
# group information about BRs by canton (count descriptive values)
nr_br_canton = council_merge.groupby("Kanton").count()[['Name']].reset_index()
nr_br_canton.columns = ['Kanton', 'NR_BR']

In [38]:
# calculate time in office for each BR (from date of election to date he/she left the office)
council_merge['time_in_office'] = (council_merge['Left office']-council_merge['Election']).astype('timedelta64[h]')

In [39]:
# sum time in office by canton
time_br_canton = council_merge[['Kanton', 'time_in_office']].groupby("Kanton").sum().reset_index()

In [40]:
# merge datasets
share_kanton = pd.merge(kantone, time_br_canton, how='left', on='Kanton')
share_kanton = pd.merge(share_kanton, nr_br_canton, how='left', on='Kanton')

In [41]:
# remove unused rows
share_kanton = share_kanton.drop(share_kanton.index[26])

In [42]:
# create weighted population column / since Jura official exists since 1979 it cannot claim as much BR seats for the time since 1848
share_kanton['size_weighted'] = share_kanton['Einwohner'].apply(int)
share_kanton.loc[share_kanton['Kanton'] == "Jura", 'size_weighted'] = int(share_kanton.loc[share_kanton['Kanton'] == "Jura", 'size_weighted'].values[0]) * 1 / (2019 - 1848) * (2019 - 1979)

In [43]:
# calculate nr of BR seats each canton would be entitled to based on its population in 2018 / given the total amount of BR
share_kanton["BR_calc"] = share_kanton.NR_BR.sum() / share_kanton['size_weighted'].sum() * share_kanton['size_weighted']
share_kanton["BR_calc"] = share_kanton["BR_calc"].round(0)

In [44]:
# calculate time in office (h) each canton would be entitled to based on its population in 2018 / given the total time spent in office
share_kanton["time_calc"] = share_kanton.time_in_office.sum() / share_kanton['size_weighted'].sum() * share_kanton['size_weighted']
share_kanton["time_calc"] = share_kanton["time_calc"].round(0)

In [45]:
# calculate difference between real values and the ones which the canton would be entitled to
share_kanton['BR_diff'] =  share_kanton['NR_BR'] - share_kanton['BR_calc']
share_kanton['time_diff'] =  (share_kanton['time_in_office'] - share_kanton['time_calc']).round(0)

In [46]:
# fix for cantons which didn't have a BR
for i, kte in share_kanton.iterrows():
    if (np.isnan(kte['BR_diff'])):
        share_kanton.loc[i, 'BR_diff'] = -kte['BR_calc']
    if (np.isnan(kte['time_diff'])):
        share_kanton.loc[i, 'time_diff'] = -kte['time_calc']

In [47]:
# calculcate mean time in office
council_merge['time_in_office'].mean() / 24

3672.6719187675067

### Regionen

In [48]:
# collect neccessary data
share_region = share_kanton[['Abk.', 'time_in_office', 'NR_BR', 'size_weighted']]
share_region.columns = ['kte_kurz', 'time_in_office', 'NR_BR', 'size_weighted']

In [49]:
# add information about the regions (based on BFS definition)
share_region.loc[share_region.kte_kurz.isin(["FR", "BE", "SO", "NE", "JU"]), "region"] = "Espace Mittelland" ## inkl. Jura
share_region.loc[share_region.kte_kurz.isin(["GE", "VD", "VS"]), "region"] = "Genferseeregion" ##inkl. NE
share_region.loc[share_region.kte_kurz.isin(["TI"]), "region"] = "Italienische Schweiz" ##ohne GR
share_region.loc[share_region.kte_kurz.isin(["ZH"]), "region"] = "Zürich"
share_region.loc[share_region.kte_kurz.isin(["GL", "SH", "AR", "AI", "SG", "GR", "TG"]), "region"] = "Ostschweiz"
share_region.loc[share_region.kte_kurz.isin(["LU", "UR", "SZ", "OW", "NW", "ZG"]), "region"] = "Zentralschweiz"
share_region.loc[share_region.kte_kurz.isin(["BS", "BL", "AG"]), "region"] = "Nordwestschweiz"

/Users/sco/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/sco/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [50]:
# sum 'time_in_office', 'NR_BR' and 'size_weighted' by region
share_region = share_region.fillna(0)
share_region = share_region.groupby("region").sum().reset_index()

In [51]:
# calculate nr of BR seats each region would be entitled to based on its population in 2018 / given the total amount of BR
share_region["BR_calc"] = share_region.NR_BR.sum() / share_region['size_weighted'].sum() * share_region['size_weighted']
share_region["BR_calc"] = share_region["BR_calc"].round(0)

In [52]:
# calculate time in office (h) each region would be entitled to based on its population in 2018 / given the total time spent in office
share_region["time_calc"] = share_region.time_in_office.sum() / share_region['size_weighted'].sum() * share_region['size_weighted']
share_region["time_calc"] = share_region["time_calc"].round(0)

In [53]:
# calculate difference between real values and the ones which the canton would be entitled to
share_region['BR_diff'] =  share_region['NR_BR'] - share_region['BR_calc']
share_region['time_diff'] =  share_region['time_in_office'] - share_region['time_calc']

,region,time_in_office,NR_BR,size_weighted,BR_calc,time_calc,BR_diff,time_diff
3,Nordwestschweiz,1063008.0,8.0,1.151919e+06,16.0,1433536.0,-8.0,-370528.0
5,Zentralschweiz,930384.0,8.0,8.060710e+05,11.0,1003137.0,-3.0,-72753.0
6,Zürich,1494106.0,20.0,1.504346e+06,21.0,1872124.0,-1.0,-378018.0
1,Genferseeregion,1773620.0,24.0,1.629841e+06,23.0,2028299.0,1.0,-254679.0
4,Ostschweiz,1552330.0,18.0,1.169358e+06,17.0,1455239.0,1.0,97091.0
2,Italienische Schweiz,700258.0,8.0,3.537090e+05,5.0,440183.0,3.0,260075.0
0,Espace Mittelland,2974724.0,33.0,1.812740e+06,26.0,2255913.0,7.0,718811.0


## Export der Daten

In [60]:
# Export information about BR since 1848
data[['year', 'BR_closeness']].to_csv("output/viz_br_parties.csv", index=False)
data[['year', 'share_female']].to_csv("output/viz_br_frauenanteil.csv", index=False)
data[['year', 'mean_age', 'age_population']].to_csv("output/viz_altersverteilung.csv", index=False)
data[['year', 'nr_family']].to_csv("output/viz_br_social_mobility.csv", index=False)
#data.to_csv("output/full_data.csv", index=False)

In [55]:
share_kanton.to_csv("output/kantone_br_verteilung.csv", index=False)

In [56]:
# Prepare data of cantons for export
## keep only neccessary columns
viz_kanton = share_kanton[["Abk.", "Kanton", "BR_diff", "time_diff"]]
## rename columns
viz_kanton.columns = ["kte_kurz", "Kanton", "BR_diff", "time_diff"]
## reformat time difference from h to d
viz_kanton["time_diff"] = (viz_kanton["time_diff"] / 24).round(0)

/Users/sco/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [59]:
# Prepare geographical data
viz_kanton[["kte_kurz", "BR_diff", "time_diff"]].to_csv("output/viz_kantone_br_bars_print.csv", index=False)
viz_kanton[["kte_kurz", "BR_diff"]].to_csv("output/viz_kantone_br_bars_online.csv", index=False)
share_kanton.to_csv("output/viz_kantone_br_map.csv", index=False)
share_region[['region', 'BR_diff']].to_csv("output/viz_regions_br_bars.csv", index=False)